# Random Forest Classification

## Importing the libraries

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

In [2]:
%cd Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\SleepDay_Processed_Files
df = pd.read_csv('SleepDayCateg_Methylome_APPRDXFilter.csv')
print(df.shape)
df.head()

Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\SleepDay_Processed_Files
(458, 10421)


,Sentrix,PATNO,HYS,Gender,APPRDX,MDSP_SleepDay,cg03441945,cg21094410,cg22926093,cg25025609,...,cg00877887,cg10042437,cg11708297,cg11562513,cg02337566,cg02550961,cg04001842,cg02454595,cg07821960,cg21267231
0,200991620021_R01C01,3001,2,1,1,1,0.908813,0.011350,0.012197,0.022372,...,0.057492,0.172612,0.036982,0.032567,0.175452,0.036156,0.115088,0.058504,0.128815,0.197503
1,200973410167_R04C01,3007,2,1,1,1,0.866976,0.022294,0.017479,0.028596,...,0.061353,0.099857,0.021763,0.024687,0.131826,0.045940,0.050066,0.042324,0.052716,0.134819
2,200973410167_R05C01,3010,2,1,1,1,0.887028,0.018926,0.014000,0.030146,...,0.049380,0.144351,0.015313,0.054307,0.142940,0.035736,0.089561,0.028921,0.039875,0.148052
3,200989060001_R06C01,3012,2,1,1,1,0.894524,0.013425,0.009698,0.015964,...,0.050630,0.122336,0.025044,0.031522,0.125793,0.028331,0.068627,0.049816,0.058885,0.127584
4,200989060143_R08C01,3024,2,1,1,1,0.931170,0.010072,0.008434,0.018157,...,0.046501,0.122477,0.018527,0.033071,0.142210,0.025472,0.068542,0.042210,0.068121,0.125470


In [3]:
df1 = df.drop(['PATNO', 'Sentrix', 'HYS', 'Gender', 'MDSP_SleepDay'], axis=1)
df1.head(2)

,APPRDX,cg03441945,cg21094410,cg22926093,cg25025609,cg16704802,cg06228737,cg00220102,cg19016494,cg12247200,...,cg00877887,cg10042437,cg11708297,cg11562513,cg02337566,cg02550961,cg04001842,cg02454595,cg07821960,cg21267231
0,1,0.908813,0.011350,0.012197,0.022372,0.018128,0.945195,0.730494,0.017911,0.921435,...,0.057492,0.172612,0.036982,0.032567,0.175452,0.036156,0.115088,0.058504,0.128815,0.197503
1,1,0.866976,0.022294,0.017479,0.028596,0.031988,0.937886,0.646004,0.075093,0.930482,...,0.061353,0.099857,0.021763,0.024687,0.131826,0.045940,0.050066,0.042324,0.052716,0.134819


In [4]:
df1 = df1.transpose()
df1.head(2)

,0,1,2,3,4,5,6,7,8,9,...,448,449,450,451,452,453,454,455,456,457
APPRDX,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,2.00000,2.00000,2.000000,2.000000,2.000000,2.000000
cg03441945,0.908813,0.866976,0.887028,0.894524,0.93117,0.909307,0.887705,0.889597,0.888017,0.914914,...,0.904465,0.883143,0.888453,0.906461,0.87849,0.93985,0.887986,0.845388,0.863326,0.859171


In [5]:
df1.rename(columns=df1.iloc[0], inplace = True)


In [6]:
df1.index.name ='CpG'
df1.reset_index(inplace=True)
df1.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
0,APPRDX,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,2.00000,2.00000,2.000000,2.000000,2.000000,2.000000
1,cg03441945,0.908813,0.866976,0.887028,0.894524,0.93117,0.909307,0.887705,0.889597,0.888017,...,0.904465,0.883143,0.888453,0.906461,0.87849,0.93985,0.887986,0.845388,0.863326,0.859171


In [7]:
df1 = df1.drop ([0], axis = 0)
df1.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,cg03441945,0.908813,0.866976,0.887028,0.894524,0.931170,0.909307,0.887705,0.889597,0.888017,...,0.904465,0.883143,0.888453,0.906461,0.878490,0.93985,0.887986,0.845388,0.863326,0.859171
2,cg21094410,0.011350,0.022294,0.018926,0.013425,0.010072,0.019717,0.013345,0.009493,0.015058,...,0.013959,0.009542,0.012160,0.011854,0.008928,0.00955,0.010887,0.012937,0.013619,0.011035


In [8]:
df2 = pd.read_csv ("SleepDay_NMI_Top50.csv")
df2.head(2)

,CpG
0,cg02449892
1,cg25993118


In [9]:
df3 = pd.merge (df1, df2, on= "CpG", how= "inner")
df3.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
0,cg26694605,0.826091,0.759600,0.75378,0.831718,0.875598,0.829044,0.846779,0.834852,0.799781,...,0.829531,0.840868,0.871458,0.824562,0.802798,0.824145,0.809247,0.797591,0.821796,0.806922
1,cg00432461,0.866254,0.850886,0.80999,0.905887,0.924254,0.903766,0.909852,0.896930,0.904402,...,0.891891,0.938651,0.902004,0.838133,0.874157,0.930700,0.881031,0.869914,0.899309,0.889846


In [10]:
df3 = df3.transpose()
df3.head(2)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
CpG,cg26694605,cg00432461,cg12483930,cg19460902,cg22274117,cg10948748,cg00077275,cg10044815,cg19559114,cg15568225,...,cg16004767,cg02039048,cg04635126,cg16517394,cg08096908,cg14478255,cg17228497,ch.8.1293020R,cg26899496,cg00111235
1.0,0.826091,0.866254,0.828487,0.792809,0.163772,0.183438,0.774203,0.894322,0.823933,0.552576,...,0.882369,0.716889,0.023837,0.092042,0.905542,0.847697,0.611259,0.288299,0.861914,0.774911


In [11]:
df3.rename(columns=df3.iloc[0], inplace = True)


In [12]:
df3.index.name ='APPRDX'
df3.reset_index(inplace=True)
df3.head(2)

,APPRDX,cg26694605,cg00432461,cg12483930,cg19460902,cg22274117,cg10948748,cg00077275,cg10044815,cg19559114,...,cg16004767,cg02039048,cg04635126,cg16517394,cg08096908,cg14478255,cg17228497,ch.8.1293020R,cg26899496,cg00111235
0,CpG,cg26694605,cg00432461,cg12483930,cg19460902,cg22274117,cg10948748,cg00077275,cg10044815,cg19559114,...,cg16004767,cg02039048,cg04635126,cg16517394,cg08096908,cg14478255,cg17228497,ch.8.1293020R,cg26899496,cg00111235
1,1.0,0.826091,0.866254,0.828487,0.792809,0.163772,0.183438,0.774203,0.894322,0.823933,...,0.882369,0.716889,0.023837,0.092042,0.905542,0.847697,0.611259,0.288299,0.861914,0.774911


In [13]:
df3 = df3.drop ([0], axis = 0)
df3.head(2)

,APPRDX,cg26694605,cg00432461,cg12483930,cg19460902,cg22274117,cg10948748,cg00077275,cg10044815,cg19559114,...,cg16004767,cg02039048,cg04635126,cg16517394,cg08096908,cg14478255,cg17228497,ch.8.1293020R,cg26899496,cg00111235
1,1.0,0.826091,0.866254,0.828487,0.792809,0.163772,0.183438,0.774203,0.894322,0.823933,...,0.882369,0.716889,0.023837,0.092042,0.905542,0.847697,0.611259,0.288299,0.861914,0.774911
2,1.0,0.7596,0.850886,0.855616,0.621401,0.072885,0.114013,0.590452,0.902732,0.882641,...,0.906843,0.716683,0.030518,0.065297,0.918975,0.874802,0.609222,0.196895,0.910184,0.844413


In [14]:
df3.reset_index(inplace = True, drop = True)
df3.head()

,APPRDX,cg26694605,cg00432461,cg12483930,cg19460902,cg22274117,cg10948748,cg00077275,cg10044815,cg19559114,...,cg16004767,cg02039048,cg04635126,cg16517394,cg08096908,cg14478255,cg17228497,ch.8.1293020R,cg26899496,cg00111235
0,1.0,0.826091,0.866254,0.828487,0.792809,0.163772,0.183438,0.774203,0.894322,0.823933,...,0.882369,0.716889,0.023837,0.092042,0.905542,0.847697,0.611259,0.288299,0.861914,0.774911
1,1.0,0.7596,0.850886,0.855616,0.621401,0.072885,0.114013,0.590452,0.902732,0.882641,...,0.906843,0.716683,0.030518,0.065297,0.918975,0.874802,0.609222,0.196895,0.910184,0.844413
2,1.0,0.75378,0.80999,0.849286,0.633239,0.064639,0.103797,0.740151,0.88532,0.848369,...,0.911231,0.747525,0.020508,0.044813,0.923476,0.800928,0.656509,0.182314,0.888291,0.829891
3,1.0,0.831718,0.905887,0.865784,0.886483,0.14063,0.07538,0.751912,0.937065,0.910269,...,0.923478,0.754439,0.030254,0.098008,0.946482,0.868114,0.763206,0.233277,0.901199,0.810589
4,1.0,0.875598,0.924254,0.856273,0.864138,0.090864,0.133332,0.803253,0.927527,0.875931,...,0.906352,0.764916,0.030531,0.083796,0.917266,0.858468,0.673198,0.284479,0.892698,0.807518


In [15]:
df3.shape

(458, 51)

In [16]:
df3 ['APPRDX'] = df3 ['APPRDX'].apply(int)


## RanFor 1000 times, CpG only 

In [17]:
df3.head(2)

,APPRDX,cg26694605,cg00432461,cg12483930,cg19460902,cg22274117,cg10948748,cg00077275,cg10044815,cg19559114,...,cg16004767,cg02039048,cg04635126,cg16517394,cg08096908,cg14478255,cg17228497,ch.8.1293020R,cg26899496,cg00111235
0,1,0.826091,0.866254,0.828487,0.792809,0.163772,0.183438,0.774203,0.894322,0.823933,...,0.882369,0.716889,0.023837,0.092042,0.905542,0.847697,0.611259,0.288299,0.861914,0.774911
1,1,0.7596,0.850886,0.855616,0.621401,0.072885,0.114013,0.590452,0.902732,0.882641,...,0.906843,0.716683,0.030518,0.065297,0.918975,0.874802,0.609222,0.196895,0.910184,0.844413


In [18]:
df3.shape

(458, 51)

In [19]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [20]:
dframe1.to_csv ("SleepDay_NMITop50CpG_Accuracy_1000RF.csv")

In [21]:
dframe1.head(2)

,Accuracy,Std_Dev
0,0.595,0.096047
1,0.695,0.117154
